In [121]:
import pickle
import torch
import numpy as np
import random

In [122]:
path_seq_val = "/export/share/krausef99dm/data/data_train/train_9.0k_data.pkl"

In [123]:
# load val data
with open(path_seq_val, 'rb') as f:
    rna_data, tissue_ids, targets, targets_bin =  pickle.load(f)

In [124]:
def compute_utr_5_lengths(rna_data):
    seq_meta = [t[:, 1] for t in rna_data]
    
    utr_5_lengths = []
    for t in seq_meta:
        unique_vals, counts = torch.unique(t, return_counts=True)
        result = dict(zip(unique_vals.tolist(), counts.tolist()))
        try:
            utr_5_lengths.append(result[5])
        except KeyError:
            utr_5_lengths.append(0)
    return utr_5_lengths

In [125]:
print(len(rna_data[120645]))
torch.unique(rna_data[120645][:, 1], return_counts=True)

7672


(tensor([1, 2, 3, 4, 5], dtype=torch.int8),
 tensor([ 846,  846,  846,  879, 4255]))

In [126]:
utr_5_lengths.index(4255)

120645

In [136]:
max_seq_len = 9000
utr_5_lengths = compute_utr_5_lengths(rna_data)
max_utr_5_len = max(utr_5_lengths)

# TODO identify global max utr_5_len
max_utr_5_len

402

In [128]:
random.seed(2)
rna_data = random.sample(rna_data, k=16)
utr_5_lengths = compute_utr_5_lengths(rna_data)

In [71]:
front_pads = [max_utr_5_len - l for l in utr_5_lengths]

In [73]:
front_pads

[514,
 595,
 639,
 477,
 578,
 526,
 620,
 601,
 463,
 576,
 578,
 477,
 514,
 628,
 513,
 415]

In [129]:
def aug_align_sequences(rna_data, max_utr_5_len):
    seq_meta = [t[:, 1] for t in rna_data]
    
    utr_5_lengths = []
    for t in seq_meta:
        unique_vals, counts = torch.unique(t, return_counts=True)
        result = dict(zip(unique_vals.tolist(), counts.tolist()))
        utr_5_lengths.append(result[5])
    
    front_pads = [max_utr_5_len - l for l in utr_5_lengths]
    
    padded_rna_data = [
        torch.cat([torch.zeros(pad, t.size(1), dtype=t.dtype), t], dim=0)
        for t, pad in zip(rna_data, front_pads)
    ]
    return padded_rna_data

In [131]:
padded_rna_data = aug_align_sequences(rna_data, max_utr_5_len)

In [135]:
# TODO Investigate further NEW max seq len
max([len(t) for t in padded_rna_data])

11535

In [82]:
idx = 0
print(rna_data[idx].shape)
print(front_pads[idx])
print(padded_rna_data[idx].shape)

torch.Size([2572, 4])
514
torch.Size([3086, 4])


In [100]:
# Verify it worked as intended
[
    (t[:, 1] == 1).nonzero(as_tuple=True)[0][0].item() if (t[:, 1] == 1).any() else -1
    for t in padded_rna_data
]

[650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650,
 650

In [ ]:
# PROBLEM: When padding to the front, total max seq length will also increase!